# Streaming

Streaming...

There are two types frequently used with Agents:


In [1]:
import * as dotenv from "dotenv/config";
import { createAgent } from "langchain";

const agent = createAgent({
    model: "openai:gpt-5-mini",
    tools: [],
    middleware: [],
    systemPrompt: "You are a full-stack comedian",
})

## No Steaming (invoke)

In [3]:
import { HumanMessage } from "langchain";

const result = await agent.invoke({
    messages: [new HumanMessage("Tell me a joke")]
})

console.log(result.messages.at(-1).content)

Why do programmers prefer dark mode?

Because light attracts bugs.

Want another one (front-end, dad, or nerdy)?


## Streaming
### `value`

In [6]:
const stream = await agent.stream({
    messages: [new HumanMessage("Tell me a joke")],
}, {
    streamMode: "values",
})

for await (const step of stream) {
    console.log(step.messages.at(-1).content)
}

Tell me a joke
Why did the full‑stack developer go broke?

Because he kept trying to cache his checks, but they kept returning null.

Want another — techy, dad, or dark (but not too dark)?


### `messages`

In [8]:
const stream = await agent.stream({
    messages: [new HumanMessage("Tell me a joke")],
}, {
    streamMode: "messages",
})

for await (const [message, metadata] of stream) {
    console.log(`[${metadata.langgraph_node}]: ${message.content}`)
}

[model_request]: 
[model_request]: Why
[model_request]:  did
[model_request]:  the
[model_request]:  developer
[model_request]:  go
[model_request]:  broke
[model_request]: ?
[model_request]:  Because
[model_request]:  he
[model_request]:  used
[model_request]:  up
[model_request]:  all
[model_request]:  his
[model_request]:  cache
[model_request]: .
[model_request]: 


In [22]:
const stream = await agent.stream({
    messages: [new HumanMessage("Write me a poem.")],
}, {
    streamMode: "messages",
})

for await (const [message] of stream) {
    console.log(message.content)
}


Stack
 Overflow
 of
 the
 Heart



I
 wake
 to
 a
 console
 of
 small
 alarms
,

syntax
 errors
 in
 the
 kitchen
 light
.

Coffee
 comp
iles
,
 steam
 in
 binary
—
warm
—

I
 run
 the
 day
 with
 a
 tentative
 git
 commit
:

"
fix
:
 optimism
;
 add
 sugar
."


My
 phone
 p
ings
 like
 a
 l
inter
 sc
olding
 me


for
 missing
 sem
icol
ons
 in
 my
 plans
.

I
 patch
 the
 morning
 with
 a
 smile
,
 import
 calm
,

from
 the
 local
 module
 labeled
 "
Maybe
."

Dependencies
:
 one
 dog
,
 two
 mism
atched
 socks
.


Con
versations
 are
 API
 calls
—

some
 return
 JSON
,
 some
 time
outs
.

I
 try
 to
 handle
 exceptions
 gently
,

catch
ing
 the
 big
 ones
 and
 logging
 the
 rest


to
 a
 file
 I
 never
 open
.


At
 noon
 I
 ref
actor
 the
 afternoon
,

pull
 requests
 of
 errands
 and
 affection
.

I
 merge
 with
 laughter
;
 resolve
 conflicts
 with
 tea
.

There
 are
 merge
 conflicts
 you
 can't
 blame
 on
 Git
:

old
 habits
,
 old
 songs
,
 the
 same
 worn
 joke
.


Even
ing
